In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.patches import Patch
from matplotlib.transforms import Bbox
from matplotlib.lines import Line2D
from matplotlib.ticker import FormatStrFormatter
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib

from scipy.stats import gamma
from scipy.stats import norm
from scipy.stats import multivariate_normal
from scipy.stats import gaussian_kde
from scipy.special import expit
from scipy.special import logit
from scipy.special import logsumexp
from scipy.optimize import minimize
from scipy.optimize import LinearConstraint
from scipy.signal import convolve
from scipy.interpolate import interp1d

from copy import deepcopy

import skfda
from skfda import FDataGrid
from skfda.exploratory.visualization import plot_fpca_perturbation_graphs
from skfda.preprocessing.dim_reduction.projection import FPCA
from skfda.representation.basis import BSpline, Fourier, Monomial

import divebomb

import time
import pickle

import importlib

matplotlib.rcParams.update({'font.size': 12})
#matplotlib.rcParams.update({'lines.markersize': 20})

np.random.seed(0)
np.set_printoptions(suppress=True,precision=4)

In [ ]:
import Preprocessor
import Parameters
import HHMM
import Visualisor

# Define Parameters

In [ ]:
train_new = False # whether to train a new model locally
iterate_models = False # whether to go through all compute canada models
find_SEs = False # refind SEs after switching states
n_points = -1 # likelihood plot resolution (-1 will throw error and stop notebook)

#model = 'CarHMM'
#model = 'HHMM'
#model = 'CarHHMM1'
model = 'CarHHMM2'

whale = '20190902-182840-CATs_OB_1'
#whale = '20200822-170001-Gikumi_CC_130'
#whale = '20200830-180909-Gikumi_CC_136'

year = whale[:4]

date = '06-14-2021'

s_dive = 0

if whale == '20190902-182840-CATs_OB_1':
    
    e_dive = 267
    stime = '2019-09-02 13:20:00'
    etime = '2019-09-02 18:00:00'
    drop_times = [[1.6,1.8],[3.3,4.3]]
    
elif whale == '20200822-170001-Gikumi_CC_130':
    
    e_dive = 10 #832
    stime = '2020-08-22 08:15:00'
    etime = '2020-08-22 15:45:00'
    drop_times = []
    
elif whale == '20200830-180909-Gikumi_CC_136':
    
    e_dive = 426
    stime = '2020-08-30 10:50:00'
    etime = '2020-08-30 16:00:00'
    drop_times = []
    HHMM_file = '../Params/CarHHMM2_k_3_3_0'
    
else:
    e_dive = 10

share_fine_states = True

HHMM_file = '../../Params/real/%s/%s/%s_%s_%s_%s' % (year,whale,whale,s_dive,e_dive,model)

In [ ]:
pars = Parameters.Parameters()

pars.cvc_file = '../Data/%s/%s/%s.cvc' % (year,whale,whale)
pars.csv_file = '../Data/%s/%s/%s.csv' % (year,whale,whale)

# only include for 2020 whale
pars.stime = stime
pars.etime = etime
pars.drop_times = drop_times

pars.share_fine_states = share_fine_states

pars.features = [{'dive_duration':{'corr':False,'f':'gamma'}},
                 {'Ax':{'corr':True,'f':'normal'},
                  'Ay':{'corr':True,'f':'normal'},
                  'Az':{'corr':True,'f':'normal'},
                  'Ahat_low':{'thresh':5,'corr':False,'f':'gamma'}}]

pars.K = [2,3]
if model == 'CarHMM':
    pars.K = [1,3]
elif model == 'HHMM':
    pars.features[1]['Ax']['corr'] = False
    pars.features[1]['Ay']['corr'] = False
    pars.features[1]['Az']['corr'] = False
elif model == 'CarHHMM1':
    pars.features[1] = {'Ax':{'corr':True,'f':'normal'},
                        'Ay':{'corr':True,'f':'normal'},
                        'Az':{'corr':True,'f':'normal'}}

prep = Preprocessor.Preprocessor(pars)

# Preprocess Data

In [ ]:
df = prep.load_data(pars.cvc_file,
                    pars.csv_file,
                    pars.cvc_cols)

In [ ]:
df = prep.prune_cols(df)

df = prep.prune_times(df,
                      pars.stime,
                      pars.etime,
                      pars.drop_times)

df = prep.fix_pressure(df)

df = prep.find_Vz(df)

df = prep.smooth_columns(df,
                         pars.smoother,
                         pars.smooth_cols)

df,dive_df = prep.find_dives(df)

data = prep.get_all_features(df,dive_df)

viz = Visualisor.Visualisor(pars,data,df)

In [ ]:
sdive = 0
edive = e_dive

fig,ax = plt.subplots(4,1,figsize = (6,6))

ax[1].yaxis.set_label_position("right")
ax[2].yaxis.set_label_position("right")
ax[3].yaxis.set_label_position("right")

temp_df = df[(df['dive_num'] >= sdive) & (df['dive_num'] <= edive)]

plt.subplot(4,1,1)
plt.plot(temp_df['hr_from_start'],temp_df['depth'],linewidth=0.8)
#plt.ylim([30,-1])
plt.gca().invert_yaxis()
plt.xticks([])
plt.axhline(0,color='k',linestyle=':',linewidth=0.5)
plt.ylabel('Depth $(m)$',fontsize=12)
plt.yticks(fontsize=12)

plt.subplot(4,1,2)
ax = plt.gca()
ax.yaxis.set_label_coords(1.06,0.5)
ax.yaxis.set_label_position("right")
ax.set_ylabel(r'$\left(y^*_{t,t^*}\right)_x$',fontsize=12,rotation=270)
plt.plot(temp_df['hr_from_start']*60,temp_df['Ax'],linewidth=1)
plt.xticks([])
#plt.ylim([-2,2])
plt.axhline(0,color='k',linestyle=':',linewidth=0.5)
#plt.yticks([-1,0,1],fontsize=12)

plt.subplot(4,1,3)
ax = plt.gca()
ax.yaxis.set_label_coords(1.06,0.5)
ax.yaxis.set_label_position("right")
ax.set_ylabel(r'$\left(y^*_{t,t^*}\right)_y$',fontsize=12,rotation=270)
plt.plot(temp_df['hr_from_start']*60,temp_df['Ay'],linewidth=1)
plt.xticks([])
#plt.ylim([-2,2])
plt.axhline(0,color='k',linestyle=':',linewidth=0.5)
#plt.yticks([-1,0,1],fontsize=12)

plt.subplot(4,1,4)
ax = plt.gca()
ax.yaxis.set_label_coords(1.06,0.5)
ax.yaxis.set_label_position("right")
ax.set_ylabel(r'$\left(y^*_{t,t^*}\right)_z$',fontsize=12,rotation=270)
plt.plot(temp_df['hr_from_start']*60,temp_df['Az'],linewidth=1)
plt.xlabel('Minute From Tagging',fontsize=12)
#plt.ylim([-2,2])
plt.axhline(0,color='k',linestyle=':',linewidth=0.5)
#plt.yticks([-1,0,1],fontsize=12)
plt.xticks(fontsize=12)

fig.text(0.025, 0.4, 'Acceleration $(m/s^2)$', va='center', rotation='vertical',fontsize=12)
plt.subplots_adjust(wspace=0, hspace=0.1)



plt.savefig('../Plots/%s/%s_%s_%s_%s_raw_data_5_dives' % (year,whale,s_dive,e_dive,model),dpi=500)
plt.show()

del temp_df

In [ ]:
fig,ax = plt.subplots(4,1,figsize = (6,6),sharex=True)
ax[1].yaxis.set_label_position("right")
ax[2].yaxis.set_label_position("right")
ax[3].yaxis.set_label_position("right")

temp_df = df

plt.subplot(4,1,1)
plt.plot(temp_df['hr_from_start'],temp_df['depth'],linewidth=0.25)
#plt.show()
#plt.title('Dive Profile / Accelerometer Data',fontsize=12)
plt.ylim([30,-1])
plt.xticks([])
plt.axhline(0,color='k',linestyle=':',linewidth=0.5)
plt.ylabel('Depth $(m)$',fontsize=12)
plt.yticks([0,10,20,30],fontsize=12)
#plt.show()

plt.subplot(4,1,2)
ax = plt.gca()
ax.yaxis.set_label_coords(1.07,0.5)
ax.yaxis.set_label_position("right")
ax.set_ylabel(r'$\left(y^*_{t,t^*}\right)_x$',fontsize=14,rotation=270)
plt.plot(temp_df['hr_from_start'],temp_df['Ax'],linewidth=0.1)
plt.xticks([])
plt.ylim([-30,30])
plt.axhline(0,color='k',linestyle=':',linewidth=0.5)
plt.yticks([-20,0,20],fontsize=12)

plt.subplot(4,1,3)
ax = plt.gca()
ax.yaxis.set_label_coords(1.07,0.5)
ax.yaxis.set_label_position("right")
ax.set_ylabel(r'$\left(y^*_{t,t^*}\right)_y$',fontsize=14,rotation=270)
plt.plot(temp_df['hr_from_start'],temp_df['Ay'],linewidth=0.1)
plt.xticks([])
plt.ylim([-30,30])
plt.axhline(0,color='k',linestyle=':',linewidth=0.5)
plt.yticks([-20,0,20],fontsize=12)

plt.subplot(4,1,4)
ax = plt.gca()
ax.yaxis.set_label_coords(1.07,0.5)
ax.yaxis.set_label_position("right")
ax.set_ylabel(r'$\left(y^*_{t,t^*}\right)_z$',fontsize=14,rotation=270)
plt.plot(temp_df['hr_from_start'],temp_df['Az'],linewidth=0.1)
plt.xlabel('Time from tag attachment $(h)$',fontsize=12)
plt.ylim([-30,30])
plt.axhline(0,color='k',linestyle=':',linewidth=0.5)
plt.yticks([-20,0,20],fontsize=12)
#plt.xticks([0,100,200,300],fontsize=12)

fig.text(0.005, 0.4, 'Acceleration $(m/s^2)$', va='center', rotation='vertical',fontsize=12)

del temp_df
plt.subplots_adjust(wspace=0, hspace=0.1)
plt.savefig('../Plots/%s/%s_%s_%s_%s_raw_data' % (year,whale,s_dive,e_dive,model),dpi=500)
plt.show()

In [ ]:
importlib.reload(Visualisor)
viz = Visualisor.Visualisor(pars,data,df)
viz.lagplot(lims={'peak_jerk':[0,50],
                  'roll_at_pj':[0,1],
                  'heading_var':[0,0.2],
                  'Ahat_low':[0,10000],
                  'FoVeDBA_high':[0,10],
                  'Vz':[-5,5],
                  'Ax':[-0.5,0.5],
                  'Ay':[0,1.0],
                  'Az':[-1.0,0.0]},
           file = '../Plots/%s/%s_%s_%s_%s_lagplot.png' % (year,whale,s_dive,e_dive,model))

In [ ]:
# plot dive duration
plt.hist([datum['dive_duration'] for datum in data],bins=50)
plt.show()

plt.hist([np.log(datum['dive_duration']) for datum in data],bins=20)
plt.show()

# get and plot acceleration
acc = []
for datum in data:
    for seg in datum['subdive_features']:
        acc.append(seg['Ax'])
        
plt.hist(acc,bins=100)
plt.show()

plt.hist([np.log(np.abs(x)) for x in acc],bins=100)
plt.show()
    
# get and plot ahat
if model != 'CarHHMM1':
    ahat = []
    for datum in data:
        for seg in datum['subdive_features']:
            ahat.append(np.log(seg['Ahat_low']))
    plt.hist(ahat,bins=100,alpha = 0.5,density=True)
    n=2
    from sklearn.mixture import BayesianGaussianMixture
    from sklearn.mixture import GaussianMixture
    GM = BayesianGaussianMixture(n,max_iter=10,n_init=10)
    GM.fit(np.array(ahat).reshape(-1, 1))
    print(GM.means_)
    print(GM.covariances_)
    print(GM.weights_)
    x = np.linspace(0,15,1000)
    total = np.zeros_like(x)
    for i in range(n):
        plt.plot(x,norm.pdf(x,loc=GM.means_[i][0],scale=GM.covariances_[i,0,0])*GM.weights_[i]/np.sum(GM.weights_))
        total += norm.pdf(x,loc=GM.means_[i][0],scale=GM.covariances_[i,0,0])*GM.weights_[i]
    plt.plot(x,total/np.sum(GM.weights_))
    plt.show()

# Explore FPCs

In [ ]:
grid_points = np.linspace(0,1.98,100)
data_matrix = []
for dive in data:
    for seg in dive['subdive_features']:
        data_matrix.append(seg['Az_all']**2 + seg['Ay_all']**2 + seg['Ax_all']**2)
        
Ax_funcs = FDataGrid(data_matrix, grid_points)
Ax_funcs.plot()

In [ ]:
Ax_funcs_fourier = Ax_funcs.to_basis(skfda.representation.basis.Fourier(n_basis=99))
Ax_funcs_fourier.plot()

In [ ]:
fpca = FPCA(n_components = 3)
fourier_coefs = fpca.fit_transform(Ax_funcs_fourier)
fpca.components_.plot()

In [ ]:
plt.plot(np.log(fourier_coefs[:,0]**2),np.log(fourier_coefs[:,1]**2),'.')
plt.show()
plt.plot(np.log(fourier_coefs[:,0]**2),np.log(fourier_coefs[:,2]**2),'.')
plt.show()
plt.plot(np.log(fourier_coefs[:,1]**2),np.log(fourier_coefs[:,2]**2),'.')

In [ ]:
for i in range(3):
    plt.hist(np.log(fourier_coefs[:,i]**2),bins=100)
    plt.show()

# Train HHMM

In [ ]:
deep_data = [data[i] for i in range(s_dive,e_dive)]
importlib.reload(HHMM)
hhmm = HHMM.HHMM(pars,deep_data)

try:
    hhmm = hhmm.load(HHMM_file)
except:
    if model == 'CarHMM':
        i = 1
    elif model == 'HHMM':
        i = 101
    elif model == 'CarHHMM1':
        i = 201
    elif model == 'CarHHMM2':
        i = 301
    else:
        print('model %s not recognized' % model)

    if model == 'CarHMM':
        HHMM_file_i = '../../Params/real/%s/%s/%s/%s_k_1_3_%s' % (year,whale,date,model,i)
    else:
        HHMM_file_i = '../../Params/real/%s/%s/%s/%s_k_2_3_%s' % (year,whale,date,model,i)
    print('file %s not found. Loading file %s' % (HHMM_file,HHMM_file_i))
    hhmm = hhmm.load(HHMM_file_i)
print(hhmm.likelihood(deep_data))

In [ ]:
if iterate_models:
    best_hhmm = hhmm
    l = hhmm.likelihood(deep_data)
    print('original likelihood:')
    print(l)
    print('')
    max_likelihood = l

    if model == 'CarHMM':
        rng = range(1,101)
    elif model == 'HHMM':
        rng = range(101,201)
    elif model == 'CarHHMM1':
        rng = range(201,301)
    elif model == 'CarHHMM2':
        rng = range(301,401)
    else:
        print('model %s not recognized' % model)

    for i in rng:

        if model == 'CarHMM':
            HHMM_file_i = '../../Params/real/%s/%s/%s/%s_k_1_3_%s' % (year,whale,date,model,i)
        else:
            HHMM_file_i = '../../Params/real/%s/%s/%s/%s_k_2_3_%s' % (year,whale,date,model,i)

        # load model
        try:
            hhmm = hhmm.load(HHMM_file_i)
        except:
            print('No existing HMM model number %d' % i)
            continue

        l = hhmm.likelihood(deep_data)
        print(l)
        if l > max_likelihood or np.isnan(max_likelihood):
            max_likelihood = l
            best_hhmm = hhmm
            print('new best model')

    hhmm = best_hhmm

In [ ]:
theta0 = deepcopy(hhmm.theta)
eta0 = deepcopy(hhmm.eta)
print(hhmm.likelihood(deep_data))
print(hhmm.theta)
print(hhmm.eta)
#hhmm.reorder_params()
print(hhmm.likelihood(deep_data))
print(hhmm.theta)
print(hhmm.eta)

In [ ]:
if train_new:
    print('Training new model')
    hhmm.train_DM(deep_data,max_steps=10,max_iters=10)
if find_SEs:
    h = 0.01
    hhmm.get_SEs(deep_data,h)
hhmm.save(HHMM_file)

In [ ]:
data,df = hhmm.label_df(data,df)

In [ ]:
w = [[],[],[],[]]
ax = [[],[],[],[]]
t = [[],[],[],[]]
starts = []
stime = data[160]['subdive_features'][0]['start_time']

for dive in data[160:166]:
    
    td = dive['subdive_features'][0]['end_time']-stime
    starts.append(td.seconds/60)
    
    for seg in dive['subdive_features']:
        state = np.argmax(seg['subdive_state_probs'])
        td = seg['start_time']-stime
        
        if model != 'CarHHMM1':
            w[state].append(seg['Ahat_low'])
        ax[state].append(seg['Ax'])
        t[state].append(td.seconds/60)
        
        if model != 'CarHHMM1':
            w[3].append(seg['Ahat_low'])
        ax[3].append(seg['Ax'])
        t[3].append(td.seconds/60)

In [ ]:
if model != 'CarHHMM1':
    colors = [cm.get_cmap('viridis')(i/(hhmm.pars.K[1]-1.0)) for i in range(hhmm.pars.K[1])]

    plt.figure(figsize=(8,3))
    plt.plot(t[3],w[3],'k--')

    for start in starts:
        plt.axvline(start,color='k')

    for i in range(3):
        plt.plot(t[i],w[i],'.',color=colors[i],markersize=10)


    plt.title('Wiggliness data for 6 dives')
    plt.ylabel(r'$\sum \hat \vec a$');
    plt.xlabel('Time (mins)')

    plt.tight_layout()
    plt.savefig('temp.png',dpi=300)

# Visualize Results

In [ ]:
print(hhmm.theta[0])
print(hhmm.theta[1][0])
print('')
print(hhmm.eta[0])
for eta0 in hhmm.eta[1]:
    print(eta0)
print('')
print(HHMM.eta_2_ptm(hhmm.eta[0]))
for eta0 in hhmm.eta[1]:
    print(HHMM.eta_2_ptm(eta0))

In [ ]:
fig = plt.figure(figsize = (5,7.5))

gs = GridSpec(3,2)

ax1=fig.add_subplot(gs[0,:])
ax2=fig.add_subplot(gs[1,0])
ax3=fig.add_subplot(gs[1,1])
ax4=fig.add_subplot(gs[2,0])
ax5=fig.add_subplot(gs[2,1])

axs = [ax2,ax3,ax4,ax5]

# Coarse Scale
            
mu = hhmm.theta[0]['dive_duration']['mu']
sig = hhmm.theta[0]['dive_duration']['sig']

dist = pars.features[0]['dive_duration']['f']
K = pars.K[0]
colors = [cm.get_cmap('tab10')(i) for i in range(K)]
legend = ['Dive %d'%(x+1) for x in range(K)]

for state in range(K):
    if dist == 'gamma':
        shape = np.square(mu)/np.square(sig)
        scale = np.square(sig)/np.array(mu)
        x = np.linspace(0.01,max(mu)+5*max(sig),100000)
        y = gamma.pdf(x,shape[state],0,scale[state])
    elif dist == 'normal':
        x = np.linspace(min(mu)-3*max(sig),max(mu)+3*max(sig),100000)
        y = norm.pdf(x,mu[state],sig[state])
    elif dist == 'vonmises':
        x = np.linspace(-np.pi,np.pi,100000)
        y = vonmises.pdf(x,sig[state],loc=mu[state])
    else:
        raise('distribution %s not recognized' % dist)
    ax1.plot(x,y,color=colors[state])
    ax1.set_ylabel('Density',fontsize=12)
    ax1.set_xlabel('$Y_t$ $(s)$')
    
legend_elements = [Line2D([0], [0], marker='o', color='w', label='Dive 1',
                       markerfacecolor=colors[0], markersize=8)]

if model != 'CarHMM':
    legend_elements.append(Line2D([0], [0], marker='o', color='w', label='Dive 2',
                                  markerfacecolor=colors[1], markersize=8))

fig.legend(handles=legend_elements,
           prop={'size': 10}, ncol=2,
           mode='expand',
           bbox_to_anchor=(0.1, 0.975, 0.9, 0.5),
           loc='lower left')

if model != "CarHHMM1":
    fine_features=['Ax','Ay','Az','Ahat_low']
else:
    fine_features=['Ax','Ay','Az']

# fine scale
for feature_num,feature in enumerate(fine_features):

    mu = hhmm.theta[1][0][feature]['mu']
    sig = hhmm.theta[1][0][feature]['sig']

    dist = pars.features[1][feature]['f']
    K = pars.K[1]
    colors = [cm.get_cmap('viridis')(i) for i in [0.,0.5,1.]]
    legend = ['Subdive %d'%(x+1) for x in range(K)]
    
    # order s

    for state in range(K):
        if dist == 'gamma':
            shape = np.square(mu)/np.square(sig)
            scale = np.square(sig)/np.array(mu)
            x = np.linspace(0.01,max(mu)+5*max(sig),100000)
            y = gamma.pdf(x,shape[state],0,scale[state])
        elif dist == 'normal':
            x = np.linspace(min(mu)-3*max(sig),max(mu)+3*max(sig),100000)
            y = norm.pdf(x,mu[state],sig[state])
        elif dist == 'vonmises':
            x = np.linspace(-np.pi,np.pi,100000)
            y = vonmises.pdf(x,sig[state],loc=mu[state])
        else:
            raise('distribution %s not recognized' % dist)
        axs[feature_num].plot(x,y,color=colors[state])
        if feature_num%2 == 0:
            axs[feature_num].set_ylabel('Density',fontsize=12)
        titles = [r'$\left(\tilde A^*_{t,\tilde t^*}\right)_x$ $(m/s^2)$',
                  r'$\left(\tilde A^*_{t,\tilde t^*}\right)_y$ $(m/s^2)$',
                  r'$\left(\tilde A^*_{t,\tilde t^*}\right)_z$ $(m/s^2)$',
                  r'$\tilde W^*_{t,\tilde t^*}$']
        title = titles[feature_num]
        axs[feature_num].set_xlabel(title,fontsize=12)
        if feature == 'Ahat_low':
            axs[feature_num].set_xscale('log')
            axs[feature_num].set_yscale('log')
            axs[feature_num].set_ylim([10e-8,10e-1])
            axs[feature_num].set_xlim([10e-1,10e4])

        legend_elements = [Line2D([0], [0], marker='o', color='w', label='Subdive 1',
                               markerfacecolor=colors[0], markersize=8),
                           Line2D([0], [0], marker='o', color='w', label='Subdive 2',
                               markerfacecolor=colors[1], markersize=8),
                           Line2D([0], [0], marker='o', color='w', label='Subdive 3',
                               markerfacecolor=colors[2], markersize=8)]

        fig.legend(handles=legend_elements,
                   prop={'size': 10}, ncol=3,
                   mode='expand',
                   bbox_to_anchor=(0.1, 0.65, 0.9, 0.5),
                   loc='lower left')

plt.tight_layout()
bbox = Bbox([[0,0],[5,8]])
file = '../Plots/%s/%s_%s_%s_%s_all_emissions.png' % (year,whale,s_dive,e_dive,model)
plt.savefig(file,bbox_inches=bbox,dpi=500)
plt.show()

In [ ]:
print(hhmm.likelihood(deep_data))

sdive = 160
edive = 165

importlib.reload(Visualisor)
viz = Visualisor.Visualisor(pars,data,df)
viz.hhmm = hhmm
#rcParams.update({'figure.autolayout': True})
viz.plot_emission_probs(0,file='../Plots/%s/%s_%s_%s_%s-coarse-emissions.png'%(year,whale,s_dive,e_dive,model))
if model == 'CarHHMM1':
    viz.plot_emission_probs(1,
                            file='../Plots/%s/%s_%s_%s_%s-fine-emissions.png' % (year,whale,s_dive,e_dive,model),
                            features=['Ax','Ay','Az'])
else:
    viz.plot_emission_probs(1,file='../Plots/%s/%s_%s_%s_%s-fine-emissions.png' % (year,whale,s_dive,e_dive,model))
viz.print_ptms()
if model != 'CarHMM':
    viz.plot_dive_features(sdive,edive,['dive_state_0_prob',
                                    'dive_state_1_prob',
                                    'subdive_state_0_prob',
                                    'subdive_state_1_prob',
                                    'subdive_state_2_prob'],[],
                           file = '../Plots/%s/%s_%s_%s_%s_decoded_states.png' % (year,whale,s_dive,e_dive,model),
                           nrows = 5)
else:
    viz.plot_dive_features(sdive,edive,['dive_state_0_prob',
                                    'subdive_state_0_prob',
                                    'subdive_state_1_prob',
                                    'subdive_state_2_prob'],[],
                           ylabs = [r'$Pr\left(X_t = 1\right)$',
                                    r'$Pr\left(\tilde X^*_{t,\tilde t^*} = 1\right)$',
                                    r'$Pr\left(\tilde X^*_{t,\tilde t^*} = 2\right)$',
                                    r'$Pr\left(\tilde X^*_{t,\tilde t^*} = 3\right)$'],
                           file = '../Plots/%s/%s_%s_%s_%s_decoded_states.png' % (year,whale,s_dive,e_dive,model),
                           nrows = 4)
    
plt.show()

viz.plot_dive_features(sdive,edive,['Ax','depth'],[],
                       file = '../Plots/%s/%s_%s_%s_%s_decoded_dives.png' % (year,whale,s_dive,e_dive,model),
                       ylabs = [r'$\left(y^*_{t,t^*}\right)_x$ $(m/s^2)$',r'Depth $(m)$'],
                       nrows = 4)

In [ ]:
'''
hhmm.theta[0]['dive_duration']['mu'] = np.array([19.481,57.330,44.278])
hhmm.theta[0]['dive_duration']['sig'] = np.array([4.798,53.621,48.755])

for i in range(3):
    hhmm.theta[1][i]['Ahat_low']['mu'] = np.array([22.385, 680.147, 5000.986])
    hhmm.theta[1][i]['Ahat_low']['sig'] = np.array([13.883, 753.257, 6079.488])
    
    hhmm.theta[1][i]['Ax']['mu'] = np.array([0.109,0.0207, -0.0752])
    hhmm.theta[1][i]['Ax']['sig'] = np.array([0.0262,0.0851,0.393])
    hhmm.theta[1][i]['Ax']['corr'] = np.array([1.705,-0.154,0.644])
    
    hhmm.theta[1][i]['Ay']['mu'] = np.array([0.271,0.169,0.160])
    hhmm.theta[1][i]['Ay']['sig'] = np.array([0.0378,0.0969,0.407])
    hhmm.theta[1][i]['Ay']['corr'] = np.array([1.705,-0.154,0.644])
    
    hhmm.theta[1][i]['Az']['mu'] = np.array([-0.0947,-0.0919,-0.411])
    hhmm.theta[1][i]['Az']['sig'] = np.array([0.0203,0.0604,0.431])
    hhmm.theta[1][i]['Az']['corr'] = np.array([1.705,-0.154,0.644])
    
hhmm.eta[0] = np.array([[0.,38.449, -8.456],
                        [1.351 , 0., -2.556],
                        [-3.319, -33.8275, 0.]])

hhmm.eta[1][0] = np.array([[0.0, -0.915, -34.204],
                           [-1.566, 0.0, -3.156],
                           [-35.718, 0.439, 0.0]])

hhmm.eta[1][1] = np.array([[0.0, -1.333, -8.064],
                           [-1.643, 0.0, -3.605],
                           [-36.476, -1.389, 0.0]])

hhmm.eta[1][2] = np.array([[0.0, -0.319, -41.542],
                           [-4.141, 0.0, -1.169],
                           [-38.222, -1.750, 0.0]])

hhmm.save(HHMM_file)
'''

# Histograms over emission distributions

In [ ]:
# colors
dive_colors = [cm.get_cmap('tab10')(i) for i in range(hhmm.pars.K[0])]
subdive_colors = [cm.get_cmap('viridis')(i/(hhmm.pars.K[1]-1.0)) for i in range(hhmm.pars.K[1])]

# dive duration
weights = [list([]) for _ in range(hhmm.pars.K[0])]
values = []

for dive in data:
    for i in range(hhmm.pars.K[0]):
        weights[i].append(dive['dive_state_probs'][i])
    values.append(dive['dive_duration'])
        
print(len(weights[0]))
        
mu = hhmm.theta[0]['dive_duration']['mu']
sig = hhmm.theta[0]['dive_duration']['sig']
shape = np.square(mu)/np.square(sig)
scale = np.square(sig)/np.array(mu)



plt.figure(figsize=(3,3))
for state in range(hhmm.pars.K[0]):
    bins = np.linspace(0.01,mu[state]+5*sig[state],25)
    x = np.linspace(0.01,mu[state]+5*sig[state],10000)
    y = gamma.pdf(x,shape[state],0,scale[state])
    plt.hist(values,weights=weights[state],density=True,bins=bins,color=dive_colors[state],alpha=0.5)
    plt.plot(x,y,color=dive_colors[state],linewidth=2)

plt.legend(['Dive type %s' % (i+1) for i in range(hhmm.pars.K[0])],fontsize=10)

for state in range(hhmm.pars.K[0]):
    x = np.linspace(0.01,mu[state]+5*sig[state],10000)
    y = gamma.pdf(x,shape[state],0,scale[state])
    plt.plot(x,y,color='k',linewidth=1)
    
plt.title('$Y_t$ by Dive Type')
plt.ylabel('Density')
plt.xlabel('Dive Duration $(s)$')
plt.tight_layout()
plt.savefig('../Plots/%s/%s_%s_%s_%s_empirical_hist_dive_duration.png' % (year,whale,s_dive,e_dive,model),
            dpi=500)
plt.show()
    
# A-hat
if model != 'CarHHMM1':
    weights = [list([]) for _ in range(hhmm.pars.K[1])]
    values = []

    for dive in data:
        for seg in dive['subdive_features']:
            weights[0].append(seg['subdive_state_probs'][0])
            weights[1].append(seg['subdive_state_probs'][1])
            weights[2].append(seg['subdive_state_probs'][2])
            values.append(seg['Ahat_low'])

    mu = hhmm.theta[1][0]['Ahat_low']['mu']
    sig = hhmm.theta[1][0]['Ahat_low']['sig']
    shape = np.square(mu)/np.square(sig)
    scale = np.square(sig)/np.array(mu)
    for state in range(hhmm.pars.K[1]):
        plt.figure(figsize=(3,3))
        bins = np.linspace(0.01,mu[state]+5*sig[state],100)
        plt.hist(values,weights=weights[state],density=True,bins=bins,color=subdive_colors[state],alpha=0.5)
        xlim = plt.gca().get_xlim()
        ylim = plt.gca().get_ylim()
        x = np.linspace(0.01,mu[state]+5*sig[state],10000)
        y = gamma.pdf(x,shape[state],0,scale[state])
        plt.plot(x,y,color='k',linewidth=1)
        plt.title(r'$\tilde W^*_{t,\tilde t^*}$, Subdive State %d'%(state+1),fontsize=12)
        plt.xlim(xlim)
        plt.xlabel('Wiggliness')
        plt.ylim(ylim)
        plt.ylabel('Density')
        #plt.legend(['Fit Distribution','Empirical Histogram'],fontsize=12)
        plt.hist(values,weights=weights[state],density=True,bins=bins,color='k',alpha=0.1)
        plt.plot(x,y,color=subdive_colors[state],linewidth=2)
        plt.plot(x,y,color='k',linewidth=1)
        plt.tight_layout()
        plt.savefig('../Plots/%s/%s_%s_%s_%s_empirical_hist_ahat_%d.png' % (year,whale,s_dive,e_dive,model,state),
            dpi=500)
        plt.show()
    
    
# Ax
for feature in ['Ax','Ay','Az']:
    print(feature)
    weights = [list([]) for _ in range(hhmm.pars.K[1])]
    values = [list([]) for _ in range(hhmm.pars.K[1])]

    mu = hhmm.theta[1][0][feature]['mu']
    sig = hhmm.theta[1][0][feature]['sig']
    if model == 'HHMM':
        corr = np.zeros(hhmm.pars.K[1])
    else:
        corr = expit(hhmm.theta[1][0][feature]['corr'])

    for state in range(hhmm.pars.K[1]):
        for dive in data:
            seg_tm1 = dive['subdive_features'][0]
            for seg in dive['subdive_features'][1:]:
                weights[state].append(seg['subdive_state_probs'][state])
                values[state].append(seg[feature] - (corr[state]*seg_tm1[feature] + (1.0-corr[state])*mu[state]))
                seg_tm1 = seg
                
    for state in range(hhmm.pars.K[1]):
        plt.figure(figsize=(3,3))
        bins = np.linspace(-5*sig[state],5*sig[state],100)
        plt.hist(values[state],weights=weights[state],density=True,bins=bins,
                 color=subdive_colors[state],alpha=0.5)
        x = np.linspace(-5*sig[state],5*sig[state],10000)
        y = norm.pdf(x,0,sig[state])
        plt.plot(x,y,color=subdive_colors[state],linewidth=2)
        plt.ylabel('Density')
        plt.xlabel('$\Delta \mathbf{a}_%s \enspace (m/s^2)$'%feature[-1])
        plt.title(r'$\left(\tilde A^*_{t,\tilde t^*}\right)_%s$, Subdive State %d'%(feature[-1],state+1),fontsize=12)
        #plt.legend(['Fit Distribution','Empirical Histogram'],fontsize=12)
        plt.plot(x,y,color='k',linewidth=1)
        plt.hist(values[state],weights=weights[state],density=True,bins=bins,
                 color='k',alpha=0.1)
        plt.tight_layout()
        plt.savefig('../Plots/%s/%s_%s_%s_%s_empirical_hist_%s_%d' % (year,whale,s_dive,e_dive,model,feature,state),
                    dpi=500)
        plt.show()

# Psuedoresiduals

In [ ]:
x = np.linspace(-5,5,1000)
y = norm.pdf(x)

# dive duration
mu = hhmm.theta[0]['dive_duration']['mu']
sig = hhmm.theta[0]['dive_duration']['sig']
shape = np.square(mu)/np.square(sig)
scale = np.square(sig)/np.array(mu)

psuedoresids = []

for dive in data:
    cdf = 0
    for state in range(hhmm.pars.K[0]):
        cdf += dive['dive_state_probs'][state] * \
               gamma.cdf(dive['dive_duration'],
                         shape[state],0,scale[state])
    if cdf >= 1.0:
        cdf = 0.9999
        
    psuedoresids.append(norm.ppf(cdf))

plt.figure(figsize=(3,3))
plt.hist(psuedoresids,density=True,bins=20,color='k',alpha=0.5)
plt.title(r'Pseudoresiduals for $Y_t$', fontsize=12)
plt.plot(x,y,color='k',linewidth=1)
plt.xlabel('Pseudoresiduals')
plt.ylabel('Density')
#plt.legend(['$\phi(\cdot)$','Pseudoresids'],fontsize=12)
plt.tight_layout()
plt.savefig('../Plots/%s/%s_%s_%s_%s_pseudresids_dive_duration.png' % (year,whale,s_dive,e_dive,model),dpi=500)
plt.show()

# A-hat
if model != 'CarHHMM1':
    mu = hhmm.theta[1][0]['Ahat_low']['mu']
    sig = hhmm.theta[1][0]['Ahat_low']['sig']
    shape = np.square(mu)/np.square(sig)
    scale = np.square(sig)/np.array(mu)

    psuedoresids = []

    for dive in data:
        for seg in dive['subdive_features']:
            cdf = 0
            for state in range(hhmm.pars.K[1]):
                cdf += seg['subdive_state_probs'][state] * \
                        gamma.cdf(seg['Ahat_low'],shape[state],0,scale[state])
            if cdf >= 1.0:
                cdf = 0.9999
            psuedoresids.append(norm.ppf(cdf))

    plt.figure(figsize=(3,3))     
    plt.hist(psuedoresids,density=True,bins=50,color='k',alpha=0.5)
    plt.title(r'Pseudoresiduals for $\tilde W^*_{t,\tilde t^*}$', fontsize=12)
    plt.plot(x,y,color='k',linewidth=1)
    plt.xlabel('Pseudoresiduals')
    plt.ylabel('Density')
    #plt.legend(['$\phi(\cdot)$','Pseudoresids'],fontsize=12)
    plt.tight_layout()
    plt.savefig('../Plots/%s/%s_%s_%s_%s_pseudresids_ahat.png' % (year,whale,s_dive,e_dive,model),dpi=500)
    plt.show() 

for feature in ['Ax','Ay','Az']:
    print(feature)
    mu = hhmm.theta[1][0][feature]['mu']
    sig = hhmm.theta[1][0][feature]['sig']
    if model == 'HHMM':
        corr = np.zeros(hhmm.pars.K[1])
    else:
        corr = expit(hhmm.theta[1][0][feature]['corr'])
    
    psuedoresids = []

    for state in range(hhmm.pars.K[1]):
        for dive in data:
            seg_tm1 = dive['subdive_features'][0]
            for seg in dive['subdive_features'][1:]:
                cdf = 0
                for state in range(hhmm.pars.K[1]):
                    mean = corr[state]*seg_tm1[feature] + (1.0-corr[state])*mu[state]
                    cdf += seg['subdive_state_probs'][state] * \
                            norm.cdf(seg[feature],mean,sig[state])
                seg_tm1 = seg
                if cdf >= 1.0:
                    cdf = 0.9999
                psuedoresids.append(norm.ppf(cdf))
                
    plt.figure(figsize=(3,3))
    plt.hist(psuedoresids,density=True,bins=50,color='k',alpha=0.5)
    plt.plot(x,y,color='k',linewidth=0.5)
    plt.title(r'Pseudoresiduals for $\left(\tilde A^*_{t,\tilde t^*}\right)_%s$'%feature[-1], fontsize=12)
    plt.xlabel('Pseudoresiduals')
    plt.ylabel('Density')
    #plt.legend(['$\phi(\cdot)$','Pseudoresids'],fontsize=10)
    plt.tight_layout()
    plt.savefig('../Plots/%s/%s_%s_%s_%s_pseudresids_%s.png' % (year,whale,s_dive,e_dive,model,feature),dpi=500)
    plt.show()   

# Make Tables

In [ ]:
if model == 'CarHHMM2':
    table = r'''
    \begin{tabular}{ccccc}
    \multirow{2}{*}{Feature}                                                       & \multirow{2}{*}{Dive type / subdive state} & \multicolumn{3}{c}{Parameter estimate}  \\
                                                                                   &                                      & $\hat \mu$    & $\hat \sigma$ & $\hat \phi$   \\ \hline
    \multirow{2}{*}{Dive Duration $(s)$ -- $Y_t$}                                  & 1                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & ---           \\
                                                                                   & 2                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & ---           \\ \hline
    \multirow{3}{*}{$x$-Acc. $(m/s^2)$ -- $\left(\Zone_{t,\tilde t^*}\right)_x$}   & 1                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ \\
                                                                                   & 2                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ \\
                                                                                   & 3                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ \\ \hline
    \multirow{3}{*}{$y$-Acc. $(m/s^2)$ -- $\left(\Zone_{t,\tilde t^*}\right)_y$}   & 1                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ \\
                                                                                   & 2                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ \\
                                                                                   & 3                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ \\ \hline
    \multirow{3}{*}{$z$-Acc. $(m/s^2)$ -- $\left(\Zone_{t,\tilde t^*}\right)_z$}   & 1                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ \\
                                                                                   & 2                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ \\
                                                                                   & 3                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ \\ \hline
    \multirow{3}{*}{Wiggliness -- $\Ztwo_{t,\tilde t^*}$}                          & 1                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & ---           \\
                                                                                   & 2                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & ---           \\
                                                                                   & 3                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & ---           \\ \hline
    \end{tabular}
    ''' % (hhmm.theta[0]['dive_duration']['mu'][0],hhmm.SEs['dive_duration']['mu'][0],
           hhmm.theta[0]['dive_duration']['sig'][0],hhmm.SEs['dive_duration']['sig'][0],
           hhmm.theta[0]['dive_duration']['mu'][1],hhmm.SEs['dive_duration']['mu'][1],
           hhmm.theta[0]['dive_duration']['sig'][1],hhmm.SEs['dive_duration']['sig'][1],
           hhmm.theta[1][0]['Ax']['mu'][0],hhmm.SEs['Ax']['mu'][0],
           hhmm.theta[1][0]['Ax']['sig'][0],hhmm.SEs['Ax']['sig'][0],
           expit(hhmm.theta[1][0]['Ax']['corr'][0]),hhmm.SEs['Ax']['corr'][0],
           hhmm.theta[1][0]['Ax']['mu'][1],hhmm.SEs['Ax']['mu'][1],
           hhmm.theta[1][0]['Ax']['sig'][1],hhmm.SEs['Ax']['sig'][1],
           expit(hhmm.theta[1][0]['Ax']['corr'][1]),hhmm.SEs['Ax']['corr'][1],
           hhmm.theta[1][0]['Ax']['mu'][2],hhmm.SEs['Ax']['mu'][2],
           hhmm.theta[1][0]['Ax']['sig'][2],hhmm.SEs['Ax']['sig'][2],
           expit(hhmm.theta[1][0]['Ax']['corr'][2]),hhmm.SEs['Ax']['corr'][2],
           hhmm.theta[1][0]['Ay']['mu'][0],hhmm.SEs['Ay']['mu'][0],
           hhmm.theta[1][0]['Ay']['sig'][0],hhmm.SEs['Ay']['sig'][0],
           expit(hhmm.theta[1][0]['Ay']['corr'][0]),hhmm.SEs['Ay']['corr'][0],
           hhmm.theta[1][0]['Ay']['mu'][1],hhmm.SEs['Ay']['mu'][1],
           hhmm.theta[1][0]['Ay']['sig'][1],hhmm.SEs['Ay']['sig'][1],
           expit(hhmm.theta[1][0]['Ay']['corr'][1]),hhmm.SEs['Ay']['corr'][1],
           hhmm.theta[1][0]['Ay']['mu'][2],hhmm.SEs['Ay']['mu'][2],
           hhmm.theta[1][0]['Ay']['sig'][2],hhmm.SEs['Ay']['sig'][2],
           expit(hhmm.theta[1][0]['Ay']['corr'][2]),hhmm.SEs['Ay']['corr'][2],
           hhmm.theta[1][0]['Az']['mu'][0],hhmm.SEs['Az']['mu'][0],
           hhmm.theta[1][0]['Az']['sig'][0],hhmm.SEs['Az']['sig'][0],
           expit(hhmm.theta[1][0]['Az']['corr'][0]),hhmm.SEs['Az']['corr'][0],
           hhmm.theta[1][0]['Az']['mu'][1],hhmm.SEs['Az']['mu'][1],
           hhmm.theta[1][0]['Az']['sig'][1],hhmm.SEs['Az']['sig'][1],
           expit(hhmm.theta[1][0]['Az']['corr'][1]),hhmm.SEs['Az']['corr'][1],
           hhmm.theta[1][0]['Az']['mu'][2],hhmm.SEs['Az']['mu'][2],
           hhmm.theta[1][0]['Az']['sig'][2],hhmm.SEs['Az']['sig'][2],
           expit(hhmm.theta[1][0]['Az']['corr'][2]),hhmm.SEs['Az']['corr'][2],
           hhmm.theta[1][0]['Ahat_low']['mu'][0],hhmm.SEs['Ahat_low']['mu'][0],
           hhmm.theta[1][0]['Ahat_low']['sig'][0],hhmm.SEs['Ahat_low']['sig'][0],
           hhmm.theta[1][0]['Ahat_low']['mu'][1],hhmm.SEs['Ahat_low']['mu'][1],
           hhmm.theta[1][0]['Ahat_low']['sig'][1],hhmm.SEs['Ahat_low']['sig'][1],
           hhmm.theta[1][0]['Ahat_low']['mu'][2],hhmm.SEs['Ahat_low']['mu'][2],
           hhmm.theta[1][0]['Ahat_low']['sig'][2],hhmm.SEs['Ahat_low']['sig'][2])

    print(table)
    
elif model == 'CarHHMM1':
    
    table = r'''
    \begin{tabular}{ccccc}
    \multirow{2}{*}{Feature}                                                       & \multirow{2}{*}{Dive / Subdive Type} & \multicolumn{3}{c}{Parameter Estimate}        \\
                                                                                   &                                      & $\hat \mu$    & $\hat \sigma$ & $\hat \phi$   \\ \hline
    \multirow{2}{*}{Dive Duration $(s)$ -- $Y_t$}                                  & 1                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & ---           \\
                                                                                   & 2                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & ---           \\ \hline
    \multirow{3}{*}{$x$-Acc. $(m/s^2)$ -- $\left(\Zone_{t,\tilde t^*}\right)_x$}   & 1                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ \\
                                                                                   & 2                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ \\
                                                                                   & 3                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ \\ \hline
    \multirow{3}{*}{$y$-Acc. $(m/s^2)$ -- $\left(\Zone_{t,\tilde t^*}\right)_y$}   & 1                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ \\
                                                                                   & 2                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ \\
                                                                                   & 3                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ \\ \hline
    \multirow{3}{*}{$z$-Acc. $(m/s^2)$ -- $\left(\Zone_{t,\tilde t^*}\right)_z$}   & 1                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ \\
                                                                                   & 2                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ \\
                                                                                   & 3                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ \\ \hline
    \end{tabular}
    ''' % (hhmm.theta[0]['dive_duration']['mu'][0],hhmm.SEs['dive_duration']['mu'][0],
           hhmm.theta[0]['dive_duration']['sig'][0],hhmm.SEs['dive_duration']['sig'][0],
           hhmm.theta[0]['dive_duration']['mu'][1],hhmm.SEs['dive_duration']['mu'][1],
           hhmm.theta[0]['dive_duration']['sig'][1],hhmm.SEs['dive_duration']['sig'][1],
           hhmm.theta[1][0]['Ax']['mu'][0],hhmm.SEs['Ax']['mu'][0],
           hhmm.theta[1][0]['Ax']['sig'][0],hhmm.SEs['Ax']['sig'][0],
           expit(hhmm.theta[1][0]['Ax']['corr'][0]),hhmm.SEs['Ax']['corr'][0],
           hhmm.theta[1][0]['Ax']['mu'][1],hhmm.SEs['Ax']['mu'][1],
           hhmm.theta[1][0]['Ax']['sig'][1],hhmm.SEs['Ax']['sig'][1],
           expit(hhmm.theta[1][0]['Ax']['corr'][1]),hhmm.SEs['Ax']['corr'][1],
           hhmm.theta[1][0]['Ax']['mu'][2],hhmm.SEs['Ax']['mu'][2],
           hhmm.theta[1][0]['Ax']['sig'][2],hhmm.SEs['Ax']['sig'][2],
           expit(hhmm.theta[1][0]['Ax']['corr'][2]),hhmm.SEs['Ax']['corr'][2],
           hhmm.theta[1][0]['Ay']['mu'][0],hhmm.SEs['Ay']['mu'][0],
           hhmm.theta[1][0]['Ay']['sig'][0],hhmm.SEs['Ay']['sig'][0],
           expit(hhmm.theta[1][0]['Ay']['corr'][0]),hhmm.SEs['Ay']['corr'][0],
           hhmm.theta[1][0]['Ay']['mu'][1],hhmm.SEs['Ay']['mu'][1],
           hhmm.theta[1][0]['Ay']['sig'][1],hhmm.SEs['Ay']['sig'][1],
           expit(hhmm.theta[1][0]['Ay']['corr'][1]),hhmm.SEs['Ay']['corr'][1],
           hhmm.theta[1][0]['Ay']['mu'][2],hhmm.SEs['Ay']['mu'][2],
           hhmm.theta[1][0]['Ay']['sig'][2],hhmm.SEs['Ay']['sig'][2],
           expit(hhmm.theta[1][0]['Ay']['corr'][2]),hhmm.SEs['Ay']['corr'][2],
           hhmm.theta[1][0]['Az']['mu'][0],hhmm.SEs['Az']['mu'][0],
           hhmm.theta[1][0]['Az']['sig'][0],hhmm.SEs['Az']['sig'][0],
           expit(hhmm.theta[1][0]['Az']['corr'][0]),hhmm.SEs['Az']['corr'][0],
           hhmm.theta[1][0]['Az']['mu'][1],hhmm.SEs['Az']['mu'][1],
           hhmm.theta[1][0]['Az']['sig'][1],hhmm.SEs['Az']['sig'][1],
           expit(hhmm.theta[1][0]['Az']['corr'][1]),hhmm.SEs['Az']['corr'][1],
           hhmm.theta[1][0]['Az']['mu'][2],hhmm.SEs['Az']['mu'][2],
           hhmm.theta[1][0]['Az']['sig'][2],hhmm.SEs['Az']['sig'][2],
           expit(hhmm.theta[1][0]['Az']['corr'][2]),hhmm.SEs['Az']['corr'][2])

    print(table)
    
elif model == 'HHMM':
    
    table = r'''
    \begin{tabular}{ccccc}
    \multirow{2}{*}{Feature}                                                       & \multirow{2}{*}{Dive / Subdive Type} & \multicolumn{3}{c}{Parameter Estimate}          \\
                                                                                   &                                      & $\hat \mu$    & $\hat \sigma$ & $\hat \phi$     \\ \hline
    \multirow{2}{*}{Dive Duration $(s)$ -- $Y_t$}                                  & 1                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & ---             \\
                                                                                   & 2                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & ---             \\ \hline
    \multirow{3}{*}{$x$-Acc. $(m/s^2)$ -- $\left(\Zone_{t,\tilde t^*}\right)_x$}   & 1                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & ---             \\
                                                                                   & 2                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & ---             \\
                                                                                   & 3                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & ---             \\ \hline
    \multirow{3}{*}{$y$-Acc. $(m/s^2)$ -- $\left(\Zone_{t,\tilde t^*}\right)_y$}   & 1                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & ---             \\
                                                                                   & 2                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & ---             \\
                                                                                   & 3                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & ---             \\ \hline
    \multirow{3}{*}{$z$-Acc. $(m/s^2)$ -- $\left(\Zone_{t,\tilde t^*}\right)_z$}   & 1                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & ---             \\
                                                                                   & 2                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & ---             \\
                                                                                   & 3                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & ---             \\ \hline
    \multirow{3}{*}{Wiggliness -- $\Ztwo_{t,\tilde t^*}$}                           & 1                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & ---             \\
                                                                                   & 2                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & ---             \\
                                                                                   & 3                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & ---             \\ \hline
    \end{tabular}
    ''' % (hhmm.theta[0]['dive_duration']['mu'][0],hhmm.SEs['dive_duration']['mu'][0],
           hhmm.theta[0]['dive_duration']['sig'][0],hhmm.SEs['dive_duration']['sig'][0],
           hhmm.theta[0]['dive_duration']['mu'][1],hhmm.SEs['dive_duration']['mu'][1],
           hhmm.theta[0]['dive_duration']['sig'][1],hhmm.SEs['dive_duration']['sig'][1],
           hhmm.theta[1][0]['Ax']['mu'][0],hhmm.SEs['Ax']['mu'][0],
           hhmm.theta[1][0]['Ax']['sig'][0],hhmm.SEs['Ax']['sig'][0],
           hhmm.theta[1][0]['Ax']['mu'][1],hhmm.SEs['Ax']['mu'][1],
           hhmm.theta[1][0]['Ax']['sig'][1],hhmm.SEs['Ax']['sig'][1],
           hhmm.theta[1][0]['Ax']['mu'][2],hhmm.SEs['Ax']['mu'][2],
           hhmm.theta[1][0]['Ax']['sig'][2],hhmm.SEs['Ax']['sig'][2],
           hhmm.theta[1][0]['Ay']['mu'][0],hhmm.SEs['Ay']['mu'][0],
           hhmm.theta[1][0]['Ay']['sig'][0],hhmm.SEs['Ay']['sig'][0],
           hhmm.theta[1][0]['Ay']['mu'][1],hhmm.SEs['Ay']['mu'][1],
           hhmm.theta[1][0]['Ay']['sig'][1],hhmm.SEs['Ay']['sig'][1],
           hhmm.theta[1][0]['Ay']['mu'][2],hhmm.SEs['Ay']['mu'][2],
           hhmm.theta[1][0]['Ay']['sig'][2],hhmm.SEs['Ay']['sig'][2],
           hhmm.theta[1][0]['Az']['mu'][0],hhmm.SEs['Az']['mu'][0],
           hhmm.theta[1][0]['Az']['sig'][0],hhmm.SEs['Az']['sig'][0],
           hhmm.theta[1][0]['Az']['mu'][1],hhmm.SEs['Az']['mu'][1],
           hhmm.theta[1][0]['Az']['sig'][1],hhmm.SEs['Az']['sig'][1],
           hhmm.theta[1][0]['Az']['mu'][2],hhmm.SEs['Az']['mu'][2],
           hhmm.theta[1][0]['Az']['sig'][2],hhmm.SEs['Az']['sig'][2],
           hhmm.theta[1][0]['Ahat_low']['mu'][0],hhmm.SEs['Ahat_low']['mu'][0],
           hhmm.theta[1][0]['Ahat_low']['sig'][0],hhmm.SEs['Ahat_low']['sig'][0],
           hhmm.theta[1][0]['Ahat_low']['mu'][1],hhmm.SEs['Ahat_low']['mu'][1],
           hhmm.theta[1][0]['Ahat_low']['sig'][1],hhmm.SEs['Ahat_low']['sig'][1],
           hhmm.theta[1][0]['Ahat_low']['mu'][2],hhmm.SEs['Ahat_low']['mu'][2],
           hhmm.theta[1][0]['Ahat_low']['sig'][2],hhmm.SEs['Ahat_low']['sig'][2])

    print(table)
    
elif model == 'CarHMM':
    
    table = r'''
    \begin{tabular}{ccccc}
    \multirow{2}{*}{Feature}                                                       & \multirow{2}{*}{Dive / Subdive Type} & \multicolumn{3}{c}{Parameter Estimate}        \\
                                                                                   &                                      & $\hat \mu$    & $\hat \sigma$ & $\hat \phi$   \\ \hline
    \multirow{1}{*}{Dive Duration $(s)$ -- $Y_t$}                                  & 1                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & ---           \\ \hline
    \multirow{3}{*}{$x$-Acc. $(m/s^2)$ -- $\left(\Zone_{t,\tilde t^*}\right)_x$}   & 1                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ \\
                                                                                   & 2                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ \\
                                                                                   & 3                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ \\ \hline
    \multirow{3}{*}{$y$-Acc. $(m/s^2)$ -- $\left(\Zone_{t,\tilde t^*}\right)_y$}   & 1                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ \\
                                                                                   & 2                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ \\
                                                                                   & 3                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ \\ \hline
    \multirow{3}{*}{$z$-Acc. $(m/s^2)$ -- $\left(\Zone_{t,\tilde t^*}\right)_z$}   & 1                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ \\
                                                                                   & 2                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ \\
                                                                                   & 3                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ \\ \hline
    \multirow{3}{*}{Wiggliness -- $\Ztwo_{t,\tilde t^*}$}                          & 1                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & ---           \\
                                                                                   & 2                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & ---           \\
                                                                                   & 3                                    & $%.3f \pm %.3f$ & $%.3f \pm %.3f$ & ---           \\ \hline
    \end{tabular}
    ''' % (hhmm.theta[0]['dive_duration']['mu'][0],hhmm.SEs['dive_duration']['mu'][0],
           hhmm.theta[0]['dive_duration']['sig'][0],hhmm.SEs['dive_duration']['sig'][0],
           hhmm.theta[1][0]['Ax']['mu'][0],hhmm.SEs['Ax']['mu'][0],
           hhmm.theta[1][0]['Ax']['sig'][0],hhmm.SEs['Ax']['sig'][0],
           expit(hhmm.theta[1][0]['Ax']['corr'][0]),hhmm.SEs['Ax']['corr'][0],
           hhmm.theta[1][0]['Ax']['mu'][1],hhmm.SEs['Ax']['mu'][1],
           hhmm.theta[1][0]['Ax']['sig'][1],hhmm.SEs['Ax']['sig'][1],
           expit(hhmm.theta[1][0]['Ax']['corr'][1]),hhmm.SEs['Ax']['corr'][1],
           hhmm.theta[1][0]['Ax']['mu'][2],hhmm.SEs['Ax']['mu'][2],
           hhmm.theta[1][0]['Ax']['sig'][2],hhmm.SEs['Ax']['sig'][2],
           expit(hhmm.theta[1][0]['Ax']['corr'][2]),hhmm.SEs['Ax']['corr'][2],
           hhmm.theta[1][0]['Ay']['mu'][0],hhmm.SEs['Ay']['mu'][0],
           hhmm.theta[1][0]['Ay']['sig'][0],hhmm.SEs['Ay']['sig'][0],
           expit(hhmm.theta[1][0]['Ay']['corr'][0]),hhmm.SEs['Ay']['corr'][0],
           hhmm.theta[1][0]['Ay']['mu'][1],hhmm.SEs['Ay']['mu'][1],
           hhmm.theta[1][0]['Ay']['sig'][1],hhmm.SEs['Ay']['sig'][1],
           expit(hhmm.theta[1][0]['Ay']['corr'][1]),hhmm.SEs['Ay']['corr'][1],
           hhmm.theta[1][0]['Ay']['mu'][2],hhmm.SEs['Ay']['mu'][2],
           hhmm.theta[1][0]['Ay']['sig'][2],hhmm.SEs['Ay']['sig'][2],
           expit(hhmm.theta[1][0]['Ay']['corr'][2]),hhmm.SEs['Ay']['corr'][2],
           hhmm.theta[1][0]['Az']['mu'][0],hhmm.SEs['Az']['mu'][0],
           hhmm.theta[1][0]['Az']['sig'][0],hhmm.SEs['Az']['sig'][0],
           expit(hhmm.theta[1][0]['Az']['corr'][0]),hhmm.SEs['Az']['corr'][0],
           hhmm.theta[1][0]['Az']['mu'][1],hhmm.SEs['Az']['mu'][1],
           hhmm.theta[1][0]['Az']['sig'][1],hhmm.SEs['Az']['sig'][1],
           expit(hhmm.theta[1][0]['Az']['corr'][1]),hhmm.SEs['Az']['corr'][1],
           hhmm.theta[1][0]['Az']['mu'][2],hhmm.SEs['Az']['mu'][2],
           hhmm.theta[1][0]['Az']['sig'][2],hhmm.SEs['Az']['sig'][2],
           expit(hhmm.theta[1][0]['Az']['corr'][2]),hhmm.SEs['Az']['corr'][2],
           hhmm.theta[1][0]['Ahat_low']['mu'][0],hhmm.SEs['Ahat_low']['mu'][0],
           hhmm.theta[1][0]['Ahat_low']['sig'][0],hhmm.SEs['Ahat_low']['sig'][0],
           hhmm.theta[1][0]['Ahat_low']['mu'][1],hhmm.SEs['Ahat_low']['mu'][1],
           hhmm.theta[1][0]['Ahat_low']['sig'][1],hhmm.SEs['Ahat_low']['sig'][1],
           hhmm.theta[1][0]['Ahat_low']['mu'][2],hhmm.SEs['Ahat_low']['mu'][2],
           hhmm.theta[1][0]['Ahat_low']['sig'][2],hhmm.SEs['Ahat_low']['sig'][2])

    print(table)

In [ ]:
if model != 'CarHMM':
    ptm = HHMM.eta_2_ptm(hhmm.eta[0])
    delta = np.array([0.5,0.5])
    for _ in range(100):
        delta = delta.dot(ptm)

    gamma0 = r'''$$\hat \Gamma = \begin{pmatrix} 
    %.3f & %.3f \\
    %.3f & %.3f
    \end{pmatrix}$$
    $$\hat \delta = \begin{pmatrix} %.3f & %.3f \end{pmatrix}$$
    '''% tuple(np.concatenate([ptm.flatten(),delta]))

    print(gamma0)

In [ ]:
ptm1 = HHMM.eta_2_ptm(hhmm.eta[1][0])
delta1 = np.array([0.3333,0.3333,0.3334])
for _ in range(100):
    delta1 = delta1.dot(ptm1)
    
if model != 'CarHMM':
    
    ptm2 = HHMM.eta_2_ptm(hhmm.eta[1][1])
    delta2 = np.array([0.3333,0.3333,0.3334])
    for _ in range(100):
        delta2 = delta2.dot(ptm2)

    gamma0 = r'''$$\hat \Gamma^{*(1)} = \begin{pmatrix} 
    %.3f & %.3f & %.3f \\
    %.3f & %.3f & %.3f \\
    %.3f & %.3f & %.3f
    \end{pmatrix}$$
    $$\hat \delta^{*(1)} = \begin{pmatrix} %.3f & %.3f & %.3f \end{pmatrix}$$

    $$\hat \Gamma^{*(2)} = \begin{pmatrix} 
    %.3f & %.3f & %.3f \\
    %.3f & %.3f & %.3f \\
    %.3f & %.3f & %.3f
    \end{pmatrix}$$
    $$\hat \delta^{*(2)} = \begin{pmatrix} %.3f & %.3f & %.3f \end{pmatrix}$$
    ''' % tuple(np.concatenate([ptm1.flatten(),delta1,ptm2.flatten(),delta2]))
    
else:
    
    gamma0 = r'''$$\hat \Gamma^{*(1)} = \begin{pmatrix} 
    %.3f & %.3f & %.3f \\
    %.3f & %.3f & %.3f \\
    %.3f & %.3f & %.3f
    \end{pmatrix}$$
    $$\hat \delta^{*(1)} = \begin{pmatrix} %.3f & %.3f & %.3f \end{pmatrix}$$
    ''' % tuple(np.concatenate([ptm1.flatten(),delta1]))
    
print(gamma0)

# Make plots of likelihood

In [ ]:
labels = {'Ahat_low': r'$\tilde W^*_{t,\tilde t^*}$',
          'Ax': r'$\left(\tilde A^*_{t,\tilde t^*}\right)_x$',
          'Ay': r'$\left(\tilde A^*_{t,\tilde t^*}\right)_y$',
          'Az': r'$\left(\tilde A^*_{t,\tilde t^*}\right)_z$',
          'dive_duration': r'$Y_t$'}

In [ ]:
hhmm = hhmm.load(HHMM_file)
orig_l = hhmm.likelihood(data)

In [ ]:
# coarse-scale theta
for feature in []:#hhmm.theta[0]:
    print('')
    print(feature)
    
    for state_num in range(pars.K[0]):
        print('')
        print(state_num)
        
        orig_mu = float(hhmm.theta[0][feature]['mu'][state_num])
        orig_sig = float(hhmm.theta[0][feature]['sig'][state_num])
        ls = np.zeros((n_points,n_points))
        
        # get ranges to iterate through
        mus = np.linspace(orig_mu - 5*hhmm.SEs[feature]['mu'][state_num],
                          orig_mu + 5*hhmm.SEs[feature]['mu'][state_num],
                          n_points)
        
        sigs = np.linspace(orig_sig - 5*hhmm.SEs[feature]['sig'][state_num],
                           orig_sig + 5*hhmm.SEs[feature]['sig'][state_num],
                           n_points)
        
        #if feature in ['Ax','Ay','Az']:
        #    mus = np.linspace(orig_mu-0.1,orig_mu+0.1,n_points)
        #    sigs = np.linspace(orig_sig*0.9,orig_sig*1.1,n_points)
        #else:
        #    mus = np.linspace(orig_mu*0.9,orig_mu*1.1,n_points)
        #    sigs = np.linspace(orig_sig*0.9,orig_sig*1.1,n_points)
        
        for i,mu in enumerate(mus):
            hhmm.theta[0][feature]['mu'][state_num] = mu
            for j,sig in enumerate(sigs):
                hhmm.theta[0][feature]['sig'][state_num] = sig
                ls[i,j] = hhmm.likelihood(data) - orig_l
                
        hhmm.theta[0][feature]['mu'][state_num] = orig_mu
        hhmm.theta[0][feature]['sig'][state_num] = orig_sig
        
        # make a heatmap
        cmap = plt.cm.viridis
        cmap.set_under()
        plt.imshow(np.rot90(ls), cmap=cmap,
                   vmin = -5, vmax = 0,
                   extent = [mus[0],mus[-1]] + [sigs[0],sigs[-1]], 
                   aspect='auto')
        plt.plot(orig_mu,orig_sig,'r*')
        c = plt.colorbar()
        c.set_label('$\Delta \log(\mathcal{L}_{HMM})$')
        plt.xlabel(r'$\mu$')
        plt.ylabel(r'$\sigma$')
        plt.title(labels[feature] + ', Dive Type ' + str(state_num+1))
        plt.tight_layout()
        plt.savefig('../Plots/%s/%s_%s_%s_%s_coarse-theta-likelihood-%s-%d.png' % (year,whale,s_dive,e_dive,model,feature,state_num+1),dpi=500)
        plt.show()

In [ ]:
# coarse-scale gamma
if False:
    orig_gamma = HHMM.eta_2_ptm(hhmm.eta[0])
    orig_eta = deepcopy(hhmm.eta[0])

    ls = np.zeros((n_points,n_points))

    # get ranges to iterate through
    gamma_12s = np.linspace(max(10e-8,orig_gamma[0,1] - 5*hhmm.SEs['Gamma_coarse'][0,1]),
                            min(1.0-10e-8,orig_gamma[0,1] + 5*hhmm.SEs['Gamma_coarse'][0,1]),
                            n_points)

    gamma_21s = np.linspace(max(10e-8,orig_gamma[1,0] - 5*hhmm.SEs['Gamma_coarse'][1,0]),
                            min(1.0-10e-8,orig_gamma[1,0] + 5*hhmm.SEs['Gamma_coarse'][1,0]),
                            n_points)

    for i,gamma_12 in enumerate(gamma_12s):
        for j,gamma_21 in enumerate(gamma_21s):
            gamma0 = np.array([[1.0-gamma_12,gamma_12],
                              [gamma_21,1.0-gamma_21]])
            hhmm.eta[0] = HHMM.ptm_2_eta(gamma0)
            ls[i,j] = hhmm.likelihood(data) - orig_l
            print(hhmm.eta[0])
            print(orig_eta)
            print(ls[i,j])
            print('')

    hhmm.eta[0] = orig_eta

    cmap = plt.cm.viridis
    cmap.set_under()
    plt.imshow(np.rot90(ls), vmin = -5, vmax = 0, cmap=cmap,
               extent = [gamma_12s[0],gamma_12s[-1]] + [gamma_21s[0],gamma_21s[-1]], 
               aspect='auto')

    gamma0 = HHMM.eta_2_ptm(hhmm.eta[0])
    plt.plot(gamma0[0,1],gamma0[1,0],'r*')
    c = plt.colorbar()
    c.set_label('$\Delta \log(\mathcal{L}_{HMM})$')
    plt.xlabel(r'$\Gamma_{1,2}$')
    plt.ylabel(r'$\Gamma_{2,1}$')
    plt.title(r'Log-likelihood vs $\Gamma$')
    plt.tight_layout()
    plt.savefig('../Plots/%s/%s_%s_%s_%s_coarse-gamma-likelihood.png' % (year,whale,s_dive,e_dive,model),dpi=500)
    plt.show()

In [ ]:
# fine-scale theta
fine_scale_theta_ls = {}
for feature in hhmm.theta[1][0]:
    
    print('')
    print(feature)
    fine_scale_theta_ls[feature] = {}
    
    for state_num in range(pars.K[1]):
        print('')
        print(state_num)
        fine_scale_theta_ls[feature][state_num] = []
        
        orig_mu = float(hhmm.theta[1][0][feature]['mu'][state_num])
        orig_sig = float(hhmm.theta[1][0][feature]['sig'][state_num])
        orig_corr = float(hhmm.theta[1][0][feature]['corr'][state_num])
        ls = np.zeros((n_points,n_points))
        
        # get ranges to iterate through
        mus = np.linspace(orig_mu - 5*hhmm.SEs[feature]['mu'][state_num],
                          orig_mu + 5*hhmm.SEs[feature]['mu'][state_num],
                          n_points)
        
        sigs = np.linspace(orig_sig - 5*hhmm.SEs[feature]['sig'][state_num],
                           orig_sig + 5*hhmm.SEs[feature]['sig'][state_num],
                           n_points)
        
        corr_low = logit(expit(orig_corr)-5*hhmm.SEs[feature]['corr'][state_num])
        corr_high = logit(expit(orig_corr)+5*hhmm.SEs[feature]['corr'][state_num])
        
        if np.isnan(corr_low):
            corr_low = -4.0
        if np.isnan(corr_high):
            corr_high = 4.0
        
        corrs = np.linspace(corr_low,corr_high,n_points)
        
        print(corrs)

        for i,mu in enumerate(mus):
            hhmm.theta[1][0][feature]['mu'][state_num] = mu
            hhmm.theta[1][1][feature]['mu'][state_num] = mu
            for j,sig in enumerate(sigs):
                hhmm.theta[1][0][feature]['sig'][state_num] = sig
                hhmm.theta[1][1][feature]['sig'][state_num] = sig
                ls[i,j] = hhmm.likelihood(data) - orig_l
                
        fine_scale_theta_ls[feature][state_num].append(ls)
                
        hhmm.theta[1][0][feature]['mu'][state_num] = orig_mu
        hhmm.theta[1][0][feature]['sig'][state_num] = orig_sig
        hhmm.theta[1][1][feature]['mu'][state_num] = orig_mu
        hhmm.theta[1][1][feature]['sig'][state_num] = orig_sig
        
        if feature in ['Ax','Ay','Az']:
            fig, axs = plt.subplots(2,2,figsize = (8,6))
            fig.delaxes(axs[0,1])
            plt.subplot(2,2,3)
        
        # make a heatmap
        cmap = plt.cm.viridis
        cmap.set_under()
        plt.imshow(np.rot90(ls), cmap=cmap,
                   vmin = -5, vmax = 0,
                   extent = [mus[0],mus[-1]] + [sigs[0],sigs[-1]], 
                   aspect='auto')
        c = plt.colorbar()
        plt.plot(orig_mu,orig_sig,'r*')
        plt.xlabel(r'$\mu$')
        plt.ylabel(r'$\sigma$')
        
        if feature not in ['Ax','Ay','Az']:
            c.set_label('$\Delta \log(\mathcal{L}_{HMM})$')
            if state_num < 2:
                plt.title(labels[feature] + ', Subdive State ' + str(state_num+1))
            else:
                plt.title(labels[feature] + ', subdive state ' + str(state_num+1))
        if feature in ['Ax','Ay','Az']:
            plt.subplot(2,2,1)
            for i,mu in enumerate(mus):
                hhmm.theta[1][0][feature]['mu'][state_num] = mu
                hhmm.theta[1][1][feature]['mu'][state_num] = mu
                for j,corr in enumerate(corrs):
                    hhmm.theta[1][0]['Ax']['corr'][state_num] = corr
                    hhmm.theta[1][1]['Ax']['corr'][state_num] = corr
                    hhmm.theta[1][0]['Ay']['corr'][state_num] = corr
                    hhmm.theta[1][1]['Ay']['corr'][state_num] = corr
                    hhmm.theta[1][0]['Az']['corr'][state_num] = corr
                    hhmm.theta[1][1]['Az']['corr'][state_num] = corr
                    ls[i,j] = hhmm.likelihood(data) - orig_l
                    
            fine_scale_theta_ls[feature][state_num].append(ls)

            hhmm.theta[1][0][feature]['mu'][state_num] = orig_mu
            hhmm.theta[1][1][feature]['mu'][state_num] = orig_mu
            hhmm.theta[1][1]['Ax']['corr'][state_num] = orig_corr
            hhmm.theta[1][0]['Ax']['corr'][state_num] = orig_corr
            hhmm.theta[1][1]['Ay']['corr'][state_num] = orig_corr
            hhmm.theta[1][0]['Ay']['corr'][state_num] = orig_corr
            hhmm.theta[1][1]['Az']['corr'][state_num] = orig_corr
            hhmm.theta[1][0]['Az']['corr'][state_num] = orig_corr
            
            cmap = plt.cm.viridis
            cmap.set_under()
            plt.imshow(np.rot90(ls), vmin = -5, vmax = 0, cmap=cmap,
                       extent = [mus[0],mus[-1]] + [corrs[0],corrs[-1]], 
                       aspect = 'auto')
            plt.plot(orig_mu,orig_corr,'r*')
            c = plt.colorbar()
            c.set_label('$\Delta \log(\mathcal{L}_{HMM})$')
            plt.xticks([])
            plt.ylabel(r'$logit(\phi)$')
            if state_num < 2:
                plt.title(labels[feature] + ', Subdive State ' + str(state_num+1))
            else:
                plt.title(labels[feature] + ', subdive state ' + str(state_num+1))

            
            plt.subplot(2,2,4)
            for i,corr in enumerate(corrs):
                hhmm.theta[1][0]['Ax']['corr'][state_num] = corr
                hhmm.theta[1][1]['Ax']['corr'][state_num] = corr
                hhmm.theta[1][0]['Ay']['corr'][state_num] = corr
                hhmm.theta[1][1]['Ay']['corr'][state_num] = corr
                hhmm.theta[1][0]['Az']['corr'][state_num] = corr
                hhmm.theta[1][1]['Az']['corr'][state_num] = corr
                for j,sig in enumerate(sigs):
                    hhmm.theta[1][0][feature]['sig'][state_num] = sig
                    hhmm.theta[1][1][feature]['sig'][state_num] = sig
                    ls[i,j] = hhmm.likelihood(data) - orig_l

            fine_scale_theta_ls[feature][state_num].append(ls)
            
            hhmm.theta[1][0][feature]['sig'][state_num] = orig_sig
            hhmm.theta[1][1][feature]['sig'][state_num] = orig_sig
            hhmm.theta[1][1]['Ax']['corr'][state_num] = orig_corr
            hhmm.theta[1][0]['Ax']['corr'][state_num] = orig_corr
            hhmm.theta[1][1]['Ay']['corr'][state_num] = orig_corr
            hhmm.theta[1][0]['Ay']['corr'][state_num] = orig_corr
            hhmm.theta[1][1]['Az']['corr'][state_num] = orig_corr
            hhmm.theta[1][0]['Az']['corr'][state_num] = orig_corr
            
            cmap = plt.cm.viridis
            cmap.set_under()
            plt.imshow(np.rot90(ls), vmin = -5, vmax = 0, cmap=cmap,
                       extent = [corrs[0],corrs[-1]] + [sigs[0],sigs[-1]], 
                       aspect='auto')
            plt.plot(orig_corr,orig_sig,'r*')
            c = plt.colorbar()
            c.set_label('$\Delta \log(\mathcal{L}_{HMM})$')
            plt.xlabel(r'$logit(\phi)$')
            plt.yticks([])
         
        plt.tight_layout()
        plt.savefig('../Plots/%s/%s_%s_%s_%s_fine-theta-likelihood-%s-%d.png' % (year,whale,s_dive,e_dive,model,feature,state_num),dpi=500)
        plt.show()
        
        with open('fine_scale_theta_ls', 'wb') as handle:
            pickle.dump(fine_scale_theta_ls, handle)

In [ ]:
# fine-scale gamma

fine_scale_gamma_ls = {}

for state_num in range(pars.K[0]):
    print(state_num)
        
    for row in range(pars.K[1]):
        print(row)
                
        cols = np.delete([0,1,2],row)
        orig_eta = deepcopy(hhmm.eta[1][state_num])
        ls = np.zeros((n_points,n_points))
        
        orig_gamma = HHMM.eta_2_ptm(hhmm.eta[1][state_num])

        # get ranges to iterate through
        if np.isinf(hhmm.SEs['Gamma_fine'][state_num][row,cols[0]]):
            se = 0.01
        else:
            se = hhmm.SEs['Gamma_fine'][state_num][row,cols[0]]
            
            
        gamma_12s = np.linspace(max(10e-8,orig_gamma[row,cols[0]] - 5*se),
                                min(1.0-10e-8,orig_gamma[row,cols[0]] + 5*se),
                                n_points)
        
        if np.isinf(hhmm.SEs['Gamma_fine'][state_num][row,cols[1]]):
            se = 0.01
        else:
            se = hhmm.SEs['Gamma_fine'][state_num][row,cols[1]]

        gamma_13s = np.linspace(max(10e-8,orig_gamma[row,cols[1]] - 5*se),
                                min(1.0-10e-8,orig_gamma[row,cols[1]] + 5*se),
                                n_points)

        for i,gamma_12 in enumerate(gamma_12s):
            for j,gamma_13 in enumerate(gamma_13s):
        
                gamma_11 = 1.0-gamma_12-gamma_13
                
                if gamma_11 > 0.0:
                    hhmm.eta[1][state_num][row,cols[0]] = np.log(gamma_12) - np.log(gamma_11)
                    hhmm.eta[1][state_num][row,cols[1]] = np.log(gamma_13) - np.log(gamma_11)
                    ls[i,j] = hhmm.likelihood(data) - orig_l
                else:
                    ls[i,j] = -np.inf
                    
                print(HHMM.eta_2_ptm(hhmm.eta[1][state_num]))
                #print(HHMM.eta_2_ptm(orig_eta))
                print(ls[i,j])
                print('')

        hhmm.eta[1][state_num] = orig_eta
        
        fine_scale_gamma_ls[state_num] = ls

        # make a heatmap
        cmap = plt.cm.viridis
        cmap.set_under()
        plt.imshow(np.rot90(ls), vmin = -5, vmax = 0, cmap=cmap,
                   extent = [gamma_12s[0],gamma_12s[-1]] + [gamma_13s[0],gamma_13s[-1]], 
                   aspect='auto')
        gamma0 = HHMM.eta_2_ptm(hhmm.eta[1][state_num])
        plt.plot(gamma0[row,cols[0]],gamma0[row,cols[1]],'r*')
        c = plt.colorbar()
        c.set_label('$\Delta \log(\mathcal{L}_{HMM})$')
        plt.xlabel(r'$\Gamma^{*(%d)}_{%d,%d}$'%(state_num+1,row+1,cols[0]+1))
        plt.ylabel(r'$\Gamma^{*(%d)}_{%d,%d}$'%(state_num+1,row+1,cols[1]+1))
        plt.title(r'Log-likelihood vs $\Gamma^{*(%d)}_{%d,\cdot}$'%(state_num+1,row+1))
        plt.tight_layout()
        plt.savefig('../Plots/%s/%s_%s_%s_%s_fine-gamma-likelihood-%d-row_%d.png' % (year,whale,s_dive,e_dive,model,state_num,row),dpi=500)
        plt.show()
        
        with open('fine_scale_gamma_ls', 'wb') as handle:
            pickle.dump(fine_scale_gamma_ls, handle)